In [22]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt
import json
from copy import deepcopy
import random
import sys
import time
from typing import List
from pprint import PrettyPrinter

pp = PrettyPrinter(indent=4)

# get an absolute path to the directory that contains parent files
project_dir = globals()["_dh"][0]
sys.path.append(os.path.normpath(os.path.join(project_dir, "..", "..", "..", "..")))

from experiments.utils.constants import (
    PIPLINES_PATH,
    NODE_PROFILING_CONFIGS_PATH,
    NODE_PROFILING_RESULTS_PATH,
)
from experiments.utils.parser import Parser

In [23]:
series = 74
experiment_id = 1
config_key_mapper = "key_config_mapper.csv"
model_name = "yolo"
series_path = os.path.join(NODE_PROFILING_RESULTS_PATH, "series", str(series))
loader = Parser(
    series_path=series_path, config_key_mapper=config_key_mapper, model_name=model_name
)
results = loader.result_processing()
key_config_df = loader.key_config_mapper()
# print(results.columns)
# print(key_config_df.columns)
# results

In [24]:
results.columns

Index(['experiment_id', 'client_to_model_avg',
       'client_to_model_p99', 'client_to_model_p50',
       'client_to_model_var', 'client_to_model_max',
       'client_to_model_min', 'model_latencies_avg',
       'model_latencies_p99', 'model_latencies_p50', 'model_latencies_var',
       'model_latencies_max', 'model_latencies_min',
       'model_to_client_avg', 'model_to_client_p99',
       'model_to_client_p50', 'model_to_client_var',
       'model_to_client_max', 'model_to_client_min',
       'start_time', 'end_time', 'duration', 'timeout_count',
       'cpu_usage_count_avg', 'cpu_usage_count_p99', 'cpu_usage_count_p50',
       'cpu_usage_count_var', 'cpu_usage_count_max', 'cpu_usage_count_min',
       'cpu_usage_rate_avg', 'cpu_usage_rate_p99', 'cpu_usage_rate_p50',
       'cpu_usage_rate_var', 'cpu_usage_rate_max', 'cpu_usage_rate_min',
       'cpu_throttled_count_avg', 'cpu_throttled_count_p99',
       'cpu_throttled_count_p50', 'cpu_throttled_count_var',
       'cpu_throttled_co

In [25]:
configs = loader.load_configs()
for config_name, config in configs.items():
    print(f"File name: {config_name}")
    pp.pprint(config)

File name: 0.yaml
{   'benchmark_duration': 1,
    'cpu_request': ['1', '2', '4', '8'],
    'data_type': 'image',
    'max_batch_size': ['1', '2', '8', '32', '64'],
    'max_batch_time': ['1'],
    'memory_request': ['10Gi'],
    'mode': 'exponential',
    'model_variants': ['yolov5n', 'yolov5s', 'yolov5m', 'yolov5l', 'yolov5x'],
    'node_name': 'yolo',
    'num_interop_threads': ['1'],
    'num_threads': ['1'],
    'pipeline_name': 'video',
    'repetition': 1,
    'replicas': [1],
    'series': 74,
    'series_meta': 'checking the effect of changing all variables under '
                   'arrival rate 1,\\n cpu type: Intel(R) Xeon(R) Gold 6126 '
                   'CPU @ 2.60GHz',
    'timeout': 1,
    'use_threading': 'True',
    'workload_config': {'load_duration': 60, 'loads_to_test': [1, 10, 20, 40]},
    'workload_type': 'static'}


In [26]:
display(key_config_df)

,experiment_id,pipeline_name,node_name,model_variant,cpu_request,memory_request,max_batch_size,max_batch_time,load,load_duration,series,series_meta,replicas,no_engine,mode,data_type,benchmark_duration
0,1,video,yolo,yolov5n,1,10Gi,1,1,1,60,74,checking the effect of changing all variables ...,1,False,exponential,image,1
1,2,video,yolo,yolov5n,1,10Gi,1,1,10,60,74,checking the effect of changing all variables ...,1,False,exponential,image,1
2,3,video,yolo,yolov5n,1,10Gi,1,1,20,60,74,checking the effect of changing all variables ...,1,False,exponential,image,1
3,4,video,yolo,yolov5n,1,10Gi,1,1,40,60,74,checking the effect of changing all variables ...,1,False,exponential,image,1
4,5,video,yolo,yolov5n,2,10Gi,1,1,1,60,74,checking the effect of changing all variables ...,1,False,exponential,image,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
367,368,video,yolo,yolov5x,8,10Gi,8,1,40,60,74,checking the effect of changing all variables ...,1,False,exponential,image,1
368,369,video,yolo,yolov5x,1,10Gi,32,1,1,60,74,checking the effect of changing all variables ...,1,False,exponential,image,1
369,370,video,yolo,yolov5x,1,10Gi,32,1,10,60,74,checking the effect of changing all variables ...,1,False,exponential,image,1
370,371,video,yolo,yolov5x,1,10Gi,32,1,20,60,74,checking the effect of changing all variables ...,1,False,exponential,image,1


In [28]:
experiment_ids = key_config_df[
    (key_config_df["model_variant"] == "yolov5x") & (key_config_df["cpu_request"] == 4)
]["experiment_id"].tolist()
metadata_columns = ["model_variant", "max_batch_size", "cpu_request", "load"]
results_columns = ["cpu_usage_count_avg", "model_latencies_avg", "throughput_max"]
output = loader.table_maker(
    experiment_ids=experiment_ids,
    metadata_columns=metadata_columns,
    results_columns=results_columns,
)
output["expected_max_throughput"] = (
    1 / output["model_latencies_avg"] * output["max_batch_size"]
)
# output.sort_values(by='throughput')
display(output)
# ax = output.plot.bar(x='max_batch_size', y=['throughput'])
# ax.set_xlabel("Max Batch Size")
# ax.set_ylabel("Throughput (RPS)")

,model_variant,max_batch_size,cpu_request,load,cpu_usage_count_avg,model_latencies_avg,throughput_max,expected_max_throughput
0,yolov5x,1,4,1,26.847100,0.352685,1.000000,2.835391
1,yolov5x,1,4,10,388.698888,0.372418,2.674283,2.685153
2,yolov5x,1,4,20,786.241805,0.368880,2.690915,2.710908
3,yolov5x,1,4,40,1693.431877,0.366172,2.708482,2.730954
4,yolov5x,2,4,1,28.395859,0.689167,1.016949,2.902056
5,yolov5x,2,4,10,307.204361,0.639324,2.945215,3.128303
6,yolov5x,2,4,20,755.086403,0.679981,2.916073,2.941261
7,yolov5x,2,4,40,1615.482699,0.709111,2.799931,2.820432
8,yolov5x,8,4,1,41.814414,1.008736,1.000000,7.930715
9,yolov5x,8,4,10,413.023757,1.907549,2.533954,4.193863


In [30]:
project_dir = globals()["_dh"][0]
sys.path.append(os.path.normpath(os.path.join(project_dir, "..", "..", "..", "..")))

from experiments.utils.constants import (
    PIPLINES_PATH,
    NODE_PROFILING_CONFIGS_PATH,
    NODE_PROFILING_RESULTS_PATH,
)
from experiments.utils.parser import Parser

series = 74
experiment_id = 1
config_key_mapper = "key_config_mapper.csv"
model_name = "yolo"
series_path = os.path.join(NODE_PROFILING_RESULTS_PATH, "series", str(series))
loader = Parser(
    series_path=series_path, config_key_mapper=config_key_mapper, model_name=model_name
)
results = loader.result_processing()
key_config_df = loader.key_config_mapper()
experiment_ids = key_config_df[
    (key_config_df["model_variant"] == "yolov5x") & (key_config_df["cpu_request"] == 4)
]["experiment_id"].tolist()
metadata_columns = ["model_variant", "max_batch_size", "cpu_request", "load"]
results_columns = ["cpu_usage_count_avg", "model_latencies_avg", "throughput_max"]
output = loader.table_maker(
    experiment_ids=experiment_ids,
    metadata_columns=metadata_columns,
    results_columns=results_columns,
)
# output['expected_max_throughput'] = 1/output['model_latencies_avg'] * output['max_batch_size']
# output.sort_values(by='throughput')
display(output)
# ax = output.plot.bar(x='max_batch_size', y=['throughput'])
# ax.set_xlabel("Max Batch Size")
# ax.set_ylabel("Throughput (RPS)")

,model_variant,max_batch_size,cpu_request,load,cpu_usage_count_avg,model_latencies_avg,throughput_max,expected_max_throughput
0,yolov5x,1,4,1,26.847100,0.352685,1.000000,2.835391
1,yolov5x,1,4,10,388.698888,0.372418,2.674283,2.685153
2,yolov5x,1,4,20,786.241805,0.368880,2.690915,2.710908
3,yolov5x,1,4,40,1693.431877,0.366172,2.708482,2.730954
4,yolov5x,2,4,1,28.395859,0.689167,1.016949,2.902056
5,yolov5x,2,4,10,307.204361,0.639324,2.945215,3.128303
6,yolov5x,2,4,20,755.086403,0.679981,2.916073,2.941261
7,yolov5x,2,4,40,1615.482699,0.709111,2.799931,2.820432
8,yolov5x,8,4,1,41.814414,1.008736,1.000000,7.930715
9,yolov5x,8,4,10,413.023757,1.907549,2.533954,4.193863
